### `sjoin` recap

Thus far we have spatially joined the permit data to the census tract data. The result is polygon geodataframe that includes:
- a row for each permit application that spatially intersects with our census tract gdf (all permits do),
- with the geometry of the census tracts in which they reside

This means :
- a sjoin can output duplicate geometries.
- we can `dissolve` duplicate geometries and
- summarize the permit data by the census geometries

### Order matters!

By default, `sjoin` output will have the geometry of the first input geodataframe.

Thus, if we change the order of our sjoin we output point geometries instead of polygon geometries.

For example, let's see what we get if we spatially join tracts to permits.

In [ ]:
# Spatially join census tract data to permit data
permits_and_tracts_gdf = gpd.sjoin(permit_parcel_point_gdf,tracts_acs_gdf)

The shape of our output geodataframe is the same.

In [ ]:
print("Permits:", permit_parcel_point_gdf.shape)
print("Tracts:" , tracts_acs_gdf.shape)
print("Tracts and permits:", tracts_and_permits_gdf.shape)
print("Permits and tracts:", permits_and_tracts_gdf.shape)

But, when we plot it we now see points.

In [ ]:
permits_and_tracts_gdf.plot()

If we look at a few rows of the geodataframe we see that it has columns from both input geodataframes

In [ ]:
permits_and_tracts_gdf.head()


With this spatial join we have augemented our permit data with census ACS data. 
- We have a row for each permit activity report and we have point geometry to map the permit locations. 
- And we have the ACS data associated with each permit location. 

We could use this output as the input to further analysis if we were interested in exploring the relationship between the permit data and one or more ACS variables.

As we did in the prior `sjoin` example, we could also use this data to aggregrate the permit data by census tract but we would not have a direct way to map the aggregated data since our output has point and not census tract geometry. So if the goal is to summarize permit data by census tract then an `sjoin` that ouputs census tract geometries would be a better way to go about it.


### Dissolve revisted

We just saw how `dissolve` is used to remove duplicate geometries. It is also commonly used to merge adjacent geometries. Let's check this out.

First let's take a look at the Alameda County census tracts.

In [ ]:
tracts_acs_gdf.plot()

If we take a look at the data we see that all rows have the same value for `COUNTYFP` or the county FIPS code, which is `001`.

In [ ]:
tracts_acs_gdf.head(2)

Let's dissolve the Alameda County census tracts by COUNTYFP.

In [ ]:
tracts_acs_gdf_dissolved = tracts_acs_gdf.dissolve(by='COUNTYFP')
tracts_acs_gdf_dissolved

In [ ]:
tracts_acs_gdf_dissolved.plot()

Because the rows for each census tract share the same county FIPS code and the census tract geometries are all nested within the county boundary, all interior polygons were dissolved leaving one boundary polygon.  Pretty cool, huh!  The `dissolve` operation is commonly used for these kind of geometric aggregation tasks because a lot of time your geographic data has more detail than you need.